In [1]:
from __future__ import annotations

In [2]:
from typing import TypedDict, get_type_hints



In [4]:
from qcodes import Parameter

In [27]:
class DictParameter(Parameter):

    def __init__(self,
                 name,
                 dicttype: Type[T],
                 converter,
                 filtered=None,
                 **kwargs):
        # TODO Need a safe way to pull out the types
        # Annotations may not cut it due to from __future__ import Annotations
        self._dicttype = dicttype
        self._converter = converter
        if filtered is None:
            self._typemap = get_type_hints(dicttype)
        else:
            self._typemap = {k: v for k, v in get_type_hints(dicttype).items() if k in filtered}
        super().__init__(name, **kwargs)


    def _from_raw_value_to_value(self, raw_value: Any
                                 ) -> T:
        if self._converter is not None:
            converted_output = self._converter(raw_value)
        else:
            converted_output = raw_value
        
        return dict((k, converted_output[k]) for k in self._typemap.keys()
                                        if k in converted_output)

    def __getitem__(self, item: Sequence[str]) -> DataClassParameter:
        return type(self)(self.name, self._dicttype, self._converter, filtered=item)

In [28]:
class MyParameter(DictParameter):

    def get_raw(self):
        return {"a": 1, "b": 10.0, "c": "ABC"} 

In [29]:
class DataDict(TypedDict):
    a: int
    b: float
    c: str

B = MyParameter("foo", DataDict, converter=None)

In [30]:
B.get()

{'a': 1, 'b': 10.0, 'c': 'ABC'}

In [31]:
a = B["a","b"]

In [36]:
a.get()

{'a': 1, 'b': 10.0}

In [37]:
B._typemap

{'a': int, 'b': float, 'c': str}

In [38]:
a._typemap

{'a': int, 'b': float}

In [39]:
get_type_hints(DataDict)

{'a': int, 'b': float, 'c': str}

* Prototype how to register this
* How to do setpoints
* Define interface in measurement for getting data with setpoints